In [14]:
%%capture
%pip install pint numpy git+https://github.com/MarkusPic/moody_chart.git

import numpy as np
import pint
ureg = pint.UnitRegistry()
import moody_chart
import pyfluids # This does the thermodynamics

In [2]:
GRAVITY = 9.81 * ureg.m / ureg.s**2

## Overall State Variables when Optimizing

* Mass flow rate (possible two up values)
* Turbine Outlet Pressure
* Compressor Outlet Pressure
* Turbine & Compressor Efficiency

# Stuff To Check
* Make condenser outlet match the CO_2 Supply

## Vertical Pipe Slices

Fluid State Vector into each slice

Forward Euler to solve for the state vector at the end of the slice

constant mass flow rate down

* Enthalpy
  + Changes by Heat Transfer
    - Constant Soil Temp given the linear
    - slice of 2m of soil
    - cond through 3 pipe layer stackup
    - convective analysis with const Ts
  + Increases by frictional losses
  

* Pressure
  + Increase by rho g delta Z
  + Decrease by frictional losses

In [3]:
def radial_thermal_conduction(inner_radius, outer_radius, conductivity):
    return np.ln(outer_radius / inner_radius) / (2 * np.pi * conductivity * (outer_radius - inner_radius))

In [4]:
def thermal_convection(velocity, radius, conductivity, prandtl, kinematic_viscosity):
    reynolds = 2 * radius * velocity / kinematic_viscosity

    if reynolds < 2300:
        nusselt = 3.66
    else:
        nusselt = 0.023 * reynolds ** 0.8 * prandtl ** 0.3
    
    return nusselt * conductivity / (2 * radius)

In [5]:
def friction_head_loss(velocity, diameter, length):
    friction_factor = 0.0

    return friction_factor * length / diameter * velocity ** 2 / 2 / GRAVITY

## Flow Through Formation

### Heat Transfer

In [13]:
A = 2 * ureg.meter ** 2
r = A ** 0.5 / np.pi
P = r * 2 * np.pi

def in_formation_heat_transfer(mass_flow, well_to_well, soil_conductivity):
    well_temp = 220 * ureg.degC

    nusselt = 450

    h = nusselt * soil_conductivity / (2 * r)

    outlet_to_inlet_ratio = np.exp(-P * well_to_well * h / (mass_flow * inlet_specifc_heat))

    return outlet_to_inlet_ratio

### Pressure Drop

In [12]:
def in_formation_pressure_drop(volumetric_flow, viscosity, well_to_well):
    perm = 1e-12 * ureg.meter ** 2
    
    return volumetric_flow * viscosity * well_to_well / A / perm

## Condensor

In [ ]:
def UA_from_cond_ops(inlet_temp, outlet_temp, cold_well_temp, mass_flow_rate):
    heat_transfer = mass_flow_rate * (inlet_enthalpy - outlet_enthalpy)

    log_mean_temp = (inlet_temp - outlet_temp) / np.ln((inlet_temp - cold_well_temp) / (outlet_temp - cold_well_temp))

    return heat_transfer / log_mean_temp

## Constants

In [6]:
dioxide_temps = np.array([10, 20]) * ureg.degC
dioxide_temps

Magnitude,[10 20]
Units,degree_Celsius


In [7]:
dioxide_pressure = 6e6 * ureg.pascals
dioxide_pressure

6000000.0 <Unit('pascal')>

In [8]:
well_dists = np.array([70, 112]) * ureg.meters
well_dists

Magnitude,[ 70 112]
Units,meter
